In [19]:
from transformers import AutoTokenizer
import pandas as pd
from nltk import sent_tokenize
import numpy as np
from experiments.Models import SentenceClassificationBert

In [7]:
tokenizer = AutoTokenizer.from_pretrained('emilyalsentzer/Bio_ClinicalBERT')

In [6]:
model = SentenceClassificationBert()
model.model.load_weights("../models/bert_sentence_classification_model")

Some layers from the model checkpoint at emilyalsentzer/Bio_ClinicalBERT were not used when initializing TFBertModel: ['nsp___cls', 'mlm___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at emilyalsentzer/Bio_ClinicalBERT.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_ids (InputLayer)      [(None, None)]               0         []                            
                                                                                                  
 attention_mask (InputLayer  [(None, None)]               0         []                            
 )                                                                                                
                                                                                                  
 tf_bert_model_1 (TFBertMod  TFBaseModelOutputWithPooli   1083102   ['input_ids[0][0]',           
 el)                         ngAndCrossAttentions(last_   72         'attention_mask[0][0]']      
                             hidden_state=(None, None,                                      

In [9]:
df = pd.read_csv("../data/emrqa_context_test.csv")
df.head()

,question,context,answer,evidence,start_token,end_token
0,what is the dosage of the medication the patie...,record #892793\n534206895 | tgmh | 71107061 | ...,adenosine,"use of beta blockade and adenosine , her arrhy...",571,573
1,what is the patient's current dose does the pa...,record #883682\n454737049 | vdmh | 87336835 | ...,150,"prior to pregnancy , history of lamivudine/epi...",641,641
2,what is the current dose of the patient's perc...,record #508921\n\n\n515984806 | jmh | 10446912...,1-2 tab,percocet 1-2 tab po q4h prn pain\n,330,333
3,what lab results does he have that are pertine...,admission date :\n2010-02-05\ndischarge date :...,a ct scan of the head,workup at the outside hospital included a ct s...,124,129
4,when was flagyl discontinued,record #499301\n697377829 | jc | 38527267 | | ...,an additional six days,instructions to complete an additional six day...,1530,1533


In [11]:
context_tokenized_key = {}
for i in df['context']:
    context_tokenized_key[i] = sent_tokenize(i)

In [28]:
def scale(x):
    if x >= 0.5:
        return 1
    return 0

In [38]:
correct = []
for i in context_tokenized_key[df["context"][0]]:
    data = tokenizer(df["question"][0], i, padding='max_length',
                           truncation=True,
                           max_length=512,
                           return_tensors='tf')
    test_x = (data['input_ids'], data['attention_mask'])
    prediction = scale(model.model.predict(test_x))
    if prediction:
        correct.append(i)
correct

1/1 [==============================] - 0s 260ms/step
[[0]]
1/1 [==============================] - 0s 271ms/step
[[0]]
1/1 [==============================] - 0s 261ms/step
[[0]]
1/1 [==============================] - 0s 257ms/step
[[0]]
1/1 [==============================] - 0s 253ms/step
[[0]]
1/1 [==============================] - 0s 251ms/step
[[0]]
1/1 [==============================] - 0s 258ms/step
[[0]]
1/1 [==============================] - 0s 251ms/step
[[0]]
1/1 [==============================] - 0s 262ms/step
[[0]]
1/1 [==============================] - 0s 255ms/step
[[0]]
1/1 [==============================] - 0s 254ms/step
[[0]]
1/1 [==============================] - 0s 253ms/step
[[0]]
1/1 [==============================] - 0s 246ms/step
[[0]]
1/1 [==============================] - 0s 253ms/step
[[0]]
1/1 [==============================] - 0s 263ms/step
[[0]]
1/1 [==============================] - 0s 263ms/step
[[0]]
1/1 [==============================] - 0s 256ms/step
[[0

KeyboardInterrupt: 

In [42]:
qs, ss, aa = [], [], []
for ind, i in df.iterrows():
    correct = []
    sentences = context_tokenized_key[i["context"]]
    questions = [i['question']] * len(sentences)
    for q, s in zip(questions, sentences):
        tokenized = tokenizer(q, s, 
                           padding='max_length',
                           truncation=True,
                           max_length=512,
                           return_tensors='tf')
        test_x = (tokenized['input_ids'], tokenized['attention_mask'])
        prediction = scale(model.model.predict(test_x, verbose=0))
        if prediction:
            correct.append(s)
    qs.append(questions[0])
    ss.append("".join(correct))
    aa.append(i['answer'])
df_with_predicted = pd.DataFrame({"question": qs, "sentence": ss, "answer": aa})

KeyboardInterrupt: 

In [ ]:
df_with_predicted.to_csv("../data/emrqa_predicted_test.csv")